In [13]:
import pandas as pd
import numpy as np

In [19]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn import metrics

In [2]:
from DBSCAN import DBSCAN as c_DBSCAN
from Agglomerative import Agglomerative as c_Agglomerative
c_KMeans = __import__('Draft K-Means').KMeans

In [3]:
df = pd.read_csv('dataset/iris.data', header=None)

In [4]:
df.head()

In [5]:
x = df.drop([4], axis=1)
y = df[4]

In [17]:
def get_cluster_mapping_to_target(x, y, label):
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(y, label))
    print("Completeness: %0.3f" % metrics.completeness_score(y, label))
    print("V-measure: %0.3f" % metrics.v_measure_score(y, label))

## DBSCAN

In [86]:
cdbscan_model = c_DBSCAN(x, 0.8, 2)
cdbscan_label = cdbscan_model.get_all()

In [88]:
cdbscan_map = get_cluster_mapping_to_target(x, y, cdbscan_label)

In [91]:
print(cdbscan_label, y.to_numpy())

In [84]:
model = DBSCAN(eps=0.8, min_samples=2).fit(x)
model.labels_

## Agglomerative

### Single

In [65]:
cagglomerative_single_model = c_Agglomerative(x, 3, 'single')
cagglomerative_single_label = cagglomerative_single_model.get_all()

In [66]:
cagglomerative_single_map = get_cluster_mapping_to_target(x, y, cagglomerative_single_label)

In [67]:
print(cagglomerative_single_label, y.to_numpy())

### Complete

In [41]:
cagglomerative_complete_model = c_Agglomerative(x, 3, 'complete')
cagglomerative_complete_label = cagglomerative_complete_model.get_all()

In [42]:
cagglomerative_complete_map = get_cluster_mapping_to_target(x, y, cagglomerative_complete_label)

In [68]:
print(cagglomerative_complete_label, y.to_numpy())

### Average

In [44]:
cagglomerative_average_model = c_Agglomerative(x, 3, 'average')
cagglomerative_average_label = cagglomerative_average_model.get_all()

In [45]:
cagglomerative_average_map = get_cluster_mapping_to_target(x, y, cagglomerative_average_label)

In [69]:
print(cagglomerative_average_label, y.to_numpy())

### Average Group

In [46]:
cagglomerative_avg_g_model = c_Agglomerative(x, 3, 'average_group')
cagglomerative_avg_g_label = cagglomerative_avg_g_model.get_all()

In [47]:
cagglomerative_avg_g_map = get_cluster_mapping_to_target(x, y, cagglomerative_avg_g_label)

In [70]:
print(cagglomerative_avg_g_label, y.to_numpy())

## KMeans

In [16]:
ckmeans_model = c_KMeans(2, 1e-8, 1000)
ckmeans_label = ckmeans_model.fit_predict(x)